# Build baseline tfrs model 

In [1]:
PROJECT_ID = 'hybrid-vertex'  # <--- TODO: CHANGE THIS
LOCATION = 'us-central1' 

In [3]:
# !pip install tensorflow-recommenders==0.6.0 --user

In [2]:
import json
import tensorflow as tf
import tensorflow_recommenders as tfrs

from google.cloud import storage

import numpy as np
import pickle as pkl
from pprint import pprint

## Create Small Dataset for testing

### features and helper functions

In [3]:
candidate_features = {
        'track_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'album_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'track_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'album_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'duration_ms_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'track_pop_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'artist_pop_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'artist_genres_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_followers_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
}

In [4]:
cont_feats = {
    'track_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'album_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'track_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'album_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'duration_ms_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'track_pop_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'artist_pop_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'artist_genres_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_followers_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    # 'pos_seed_track': tf.io.FixedLenFeature(dtype=tf.int64, shape=()),
    'track_name_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_name_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'album_name_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'track_uri_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_uri_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'album_uri_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'duration_seed_track': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'track_pop_seed_track': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'artist_pop_seed_track': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'artist_genres_seed_track': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_followers_seed_track': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    # 'pid': tf.io.FixedLenFeature(dtype=tf.int64, shape=()),
    'name': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'collaborative': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    # 'duration_ms_seed_pl': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'n_songs_pl': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'num_artists_pl': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'num_albums_pl': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'description_pl': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
}
    ###ragged

seq_feats = {
    'track_name_pl': tf.io.RaggedFeature(tf.string),
    'artist_name_pl': tf.io.RaggedFeature(tf.string),
    'album_name_pl': tf.io.RaggedFeature(tf.string),
    'track_uri_pl': tf.io.RaggedFeature(tf.string),
    'duration_ms_songs_pl': tf.io.RaggedFeature(tf.float32),
    'artist_pop_pl': tf.io.RaggedFeature(tf.float32),
    'artists_followers_pl': tf.io.RaggedFeature(tf.float32),
    'track_pop_pl': tf.io.RaggedFeature(tf.float32),
    'artist_genres_pl': tf.io.RaggedFeature(tf.string),
}


### Playlist dataset

In [40]:
## testing output
from google.cloud import storage

client = storage.Client()
# # gs://spotify-beam-v3/v3/candidates/*.tfrecords

BUCKET = 'spotify-beam-v3'
CANDIDATE_PREFIX = 'v3/train/'

valid_files = []
for blob in client.list_blobs(f"{BUCKET}", prefix=f'{CANDIDATE_PREFIX}', delimiter="/"):
    valid_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))
    
raw_dataset = tf.data.TFRecordDataset(valid_files)

BUCKET = 'spotify-beam-v3'
CANDIDATE_PREFIX = 'v3/valid/'

valid_files = []
for blob in client.list_blobs(f"{BUCKET}", prefix=f'{CANDIDATE_PREFIX}', delimiter="/"):
    valid_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))
    
raw_dataset_valid = tf.data.TFRecordDataset(valid_files)

def parse_tfrecord(example):
    example = tf.io.parse_single_sequence_example(
        example, 
        context_features=cont_feats,
        sequence_features=seq_feats
    )
    return example

In [84]:

parsed_dataset = raw_dataset.map(parse_tfrecord)
parsed_dataset_valid = raw_dataset_valid.map(parse_tfrecord)


MAX_PLAYLIST_LENGTH = 375
MAX_TOKENS = 1_000_000

# gives: 
# array([[ 1,  2, -1, -1],
#       [ 3,  4, -1, -1]], dtype=int32)

def pad_up_to(t, max_in_dims=[1 ,MAX_PLAYLIST_LENGTH], constant_value=''):
    s = tf.shape(t)
    paddings = [[0, m-s[i]] for (i,m) in enumerate(max_in_dims)]
    return tf.pad(t, paddings, 'CONSTANT', constant_values=constant_value)

def return_padded_tensors(context, data):
    
        a = data['track_name_pl'].to_tensor(default_value='', shape=[None, MAX_PLAYLIST_LENGTH]), 
        b = data['artist_name_pl'].to_tensor(default_value='', shape=[None, MAX_PLAYLIST_LENGTH]), 
        c = data['album_name_pl'].to_tensor(default_value='', shape=[None, MAX_PLAYLIST_LENGTH]), 
        d = data['track_uri_pl'].to_tensor(default_value='', shape=[None, MAX_PLAYLIST_LENGTH]), 
        e = data['duration_ms_songs_pl'].to_tensor(default_value=-1., shape=[None, MAX_PLAYLIST_LENGTH]), 
        f = data['artist_pop_pl'].to_tensor(default_value=-1., shape=[None, MAX_PLAYLIST_LENGTH]), 
        g = data['artists_followers_pl'].to_tensor(default_value=-1., shape=[None, MAX_PLAYLIST_LENGTH]), 
        h = data['track_pop_pl'].to_tensor(default_value=-1., shape=[None, MAX_PLAYLIST_LENGTH]), 
        i = data['artist_genres_pl'].to_tensor(default_value='', shape=[None, MAX_PLAYLIST_LENGTH]), 
        
        padded_data = context.copy()
        padded_data['track_name_pl'] = a
        padded_data['artist_name_pl'] = b
        padded_data['album_name_pl'] = c
        padded_data['track_uri_pl'] = d
        padded_data['duration_ms_songs_pl'] = e
        padded_data['artist_pop_pl'] = f
        padded_data['artists_followers_pl'] = g
        padded_data['track_pop_pl'] = h
        padded_data['artist_genres_pl'] = i
        
        return padded_data
parsed_dataset_padded = parsed_dataset.map(return_padded_tensors)   
parsed_dataset_padded_valid = parsed_dataset_valid.map(return_padded_tensors)   

In [7]:
for features in parsed_dataset_padded_valid.skip(3).take(1):
    pprint(features)
    print("_______________")

{'album_name_can': <tf.Tensor: shape=(), dtype=string, numpy=b'Word Of Mouth'>,
 'album_name_pl': (<tf.Tensor: shape=(1, 375), dtype=string, numpy=
array([[b'Stranglehold', b'Roadhouse 01', b'A Song For Every Moon',
        b'Uncomfortable - Single', b'A Song For Every Moon', b'Freudian',
        b'Michl', b'Salt', b'blkswn', b'blkswn', b'blkswn', b'blkswn',
        b'Blonde', b'Chanel', b'Kiddo', b'I Learnt Some Jazz Today',
        b"Say It Here, While It's Safe",
        b'Piece of Cake (feat. Harrison Sands and Shota)',
        b'Land of Lights', b'Blonde', b'Nat Love', b'Pine & Ginger',
        b'She Say', b'S!ck S!ck S!ck', b'Telefone', b'Nocturnal',
        b'Hotel Allan', b'alarm (prod. no sentences)',
        b'Buttermilk - EP', b'Restoration of an American Idol',
        b'Into the Flame', b'The Thirst (feat. Shiz)', b'Urban Flora',
        b'A Song For Every Moon', b"Sex n' Drugs", b'Thoughts of You',
        b"Searchin'", b'Tell Me Why', b'Kiddo', b'Por Vida', b'grey',
    

### Candidate Track dataset

In [42]:
BUCKET = 'spotify-beam-v3'
CANDIDATE_PREFIX = 'v3/candidates/'

candidate_files = []
for blob in client.list_blobs(f"{BUCKET}", prefix=f'{CANDIDATE_PREFIX}', delimiter="/"):
    candidate_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))
    
candidate_dataset = tf.data.TFRecordDataset(candidate_files)

def parse_candidate_tfrecord_fn(example):
    example = tf.io.parse_single_example(
        example, 
        features=candidate_features
    )
    return example

# parsed_candidate_dataset = candidate_dataset.map(parse_candidate_tfrecord_fn, num_parallel_calls=-1)

parsed_candidate_dataset = candidate_dataset.map(parse_candidate_tfrecord_fn) ### THIS NEEDS TO BE FIXED SO THE UNIQUE PRODUCT DATASET HAS THE SAME FIELD NAMES (goes thru the same model)

In [9]:
for features in parsed_candidate_dataset.take(1):
    pprint(features)
    print("_______________")

{'album_name_can': <tf.Tensor: shape=(), dtype=string, numpy=b'The Sound of Everything Rmx'>,
 'album_uri_can': <tf.Tensor: shape=(), dtype=string, numpy=b'spotify:album:4a8tMD6qq6GUuUwNae38VI'>,
 'artist_followers_can': <tf.Tensor: shape=(), dtype=float32, numpy=277649.0>,
 'artist_genres_can': <tf.Tensor: shape=(), dtype=string, numpy=b"'downtempo', 'electronica', 'funk', 'latin alternative', 'nu jazz', 'nu-cumbia', 'trip hop', 'world'">,
 'artist_name_can': <tf.Tensor: shape=(), dtype=string, numpy=b'Quantic'>,
 'artist_pop_can': <tf.Tensor: shape=(), dtype=float32, numpy=64.0>,
 'artist_uri_can': <tf.Tensor: shape=(), dtype=string, numpy=b'spotify:artist:5ZMwoAjeDtLJ0XRwRTgaK8'>,
 'duration_ms_can': <tf.Tensor: shape=(), dtype=float32, numpy=267130.0>,
 'track_name_can': <tf.Tensor: shape=(), dtype=string, numpy=b'The Sound of Everything - Watch TV & Se\xc3\xb1orlobo Remix'>,
 'track_pop_can': <tf.Tensor: shape=(), dtype=float32, numpy=53.0>,
 'track_uri_can': <tf.Tensor: shape=(),

### Vocab files

In [10]:
BUCKET_NAME = 'spotify-v1'
FILE_PATH = 'vocabs/v1_string_vocabs'
FILE_NAME = 'string_vocabs_v1_20220705-202905.txt'
DESTINATION_FILE = 'downloaded_vocabs.txt'

client = storage.Client()

with open(f'{DESTINATION_FILE}', 'wb') as file_obj:
    client.download_blob_to_file(
        f'gs://{BUCKET_NAME}/{FILE_PATH}/{FILE_NAME}', file_obj)

    
with open(f'{DESTINATION_FILE}', 'rb') as pickle_file:
    vocab_dict_load = pkl.load(pickle_file)

In [11]:
# len(vocab_dict_load["unique_pids"])

avg_duration_ms_seed_pl = 13000151.68
var_duration_ms_seed_pl = 133092900971233.58
vocab_dict_load['avg_duration_ms_seed_pl']=avg_duration_ms_seed_pl
vocab_dict_load['var_duration_ms_seed_pl']=var_duration_ms_seed_pl

avg_n_songs_pl = 55.21
var_n_songs_pl = 2317.54
vocab_dict_load['avg_n_songs_pl']=avg_n_songs_pl
vocab_dict_load['var_n_songs_pl']=var_n_songs_pl

avg_n_artists_pl = 30.56
var_n_artists_pl = 769.26
vocab_dict_load['avg_n_artists_pl']=avg_n_artists_pl
vocab_dict_load['var_n_artists_pl']=var_n_artists_pl

avg_n_albums_pl = 40.25
var_n_albums_pl = 1305.54
vocab_dict_load['avg_n_albums_pl']=avg_n_albums_pl
vocab_dict_load['var_n_albums_pl']=var_n_albums_pl

avg_artist_pop = 16.08
var_artist_pop = 300.64
vocab_dict_load['avg_artist_pop']=avg_artist_pop
vocab_dict_load['var_artist_pop']=var_artist_pop

avg_duration_ms_songs_pl = 234823.14
var_duration_ms_songs_pl = 5558806228.41
vocab_dict_load['avg_duration_ms_songs_pl']=avg_duration_ms_songs_pl
vocab_dict_load['var_duration_ms_songs_pl']=var_duration_ms_songs_pl

avg_artist_followers = 43337.77
var_artist_followers = 377777790193.57
vocab_dict_load['avg_artist_followers']=avg_artist_followers
vocab_dict_load['var_artist_followers']=var_artist_followers

avg_track_pop = 10.85
var_track_pop = 202.18
vocab_dict_load['avg_track_pop']=avg_track_pop
vocab_dict_load['var_track_pop']=var_track_pop
# vocab_dict_load['unique_pids_string']

In [12]:
vocab_dict_load['unique_pids']

array([     0,      1,      2, ..., 999997, 999998, 999999])

In [13]:
VERSION='test-v1'

In [14]:
BUCKET='spotify-tfrecords-blog'
bucket=client.bucket(BUCKET)
blob = bucket.blob(f'vocabs_stats/vocab_dict_{VERSION}.txt')
pickle_out = pkl.dumps(vocab_dict_load)
blob.upload_from_string(pickle_out)

In [15]:
# bucket=client.bucket(BUCKET)  
# blob = bucket.blob('vocabs/string_vocabs')
# blob.upload_from_filename('string_vocabs')

### Test instances

In [16]:
playlist_test_instancet = {
    'name': np.asarray([b'Best Christmas']),
    'collaborative': np.asarray([b'false']),
    'pid': np.asarray([173671]),
    'description_pl': np.asarray([b'test description']),
    'duration_ms_seed_pl': np.asarray([5458995.]),
    'n_songs_pl': np.asarray([58.]),
    'num_artists_pl': np.asarray([19.]),
    'num_albums_pl': np.asarray([27.]),
    'artist_name_pl': np.asarray([[b'Juan Luis Guerra 4.40', b'Prince Royce', b'Luis Vargas']]),
    'track_uri_pl': np.asarray([[b'spotify:track:1g0IBPZTRP7VYkctJ4Qafg',b'spotify:track:43wUzbYxEFoXugYkgTzMWp']]),
    'track_name_pl': np.asarray([[b'Lover Come Back', b'White Lightning', b'Shake Me Down']]),
    'duration_ms_songs_pl': np.asarray([[245888., 195709., 283906., 271475., 300373., 275173., 236145.,]]),
    'album_name_pl': np.asarray([[b'Silsulim', b'Sara Shara', b'Muzika Vesheket', b'Ba La Lirkod']]),
    'artist_pop_pl': np.asarray([[81., 81., 70., 66., 66., 66., 46., 87.]]),
    'artists_followers_pl': np.asarray([[3.556710e+05, 8.200000e+02, 1.510000e+02, 1.098080e+05,]]),
    'artist_genres_pl': np.asarray([[b"'israeli pop', 'jewish pop'", b"'israeli pop', 'jewish pop'",]]),
    'track_pop_pl': np.asarray([[70, 77, 50, 44, 30, 28, 15, 26, 15, 18, 46, 38,]])
}

In [17]:
candidate_test_instance = {
    'artist_name': np.asarray([b'Hellogoodbye']),
    'track_name': np.asarray([b'When We First Met']),
    'album_name': np.asarray([b'Would It Kill You?']),
    'track_uri': np.asarray([b'Ba La Lirkod']),
    'artist_uri': np.asarray([b'spotify:artist:6GH0NzpthMGxu1mcfAkOde']),
    'album_uri': np.asarray([b'spotify:album:4dHXV7pJs6d8N9ACAMzhIw']),
    'duration_ms': np.asarray([154813.0]),
    'track_pop': np.asarray([45.0]),
    'artist_pop': np.asarray([51.0]),
    'artist_followers':np.asarray([205331.0]),
    'artist_genres': np.asarray([b"'neon pop punk', 'pop punk'"]),
    # 'test': np.asarray([b'test'])
}

# candidate_test_instance = {
#     'artist_name': np.asarray([b'Hellogoodbye']),
#     'track_name': np.asarray([b'When We First Met']),
#     'album_name': np.asarray([b'Would It Kill You?']),
#     'track_uri': np.asarray([b'Ba La Lirkod']),
#     'artist_uri': np.asarray([b'spotify:artist:6GH0NzpthMGxu1mcfAkOde']),
#     'album_uri': np.asarray([b'spotify:album:4dHXV7pJs6d8N9ACAMzhIw']),
#     'duration_ms': np.asarray([154813.0]),
#     'track_pop': np.asarray([45.0]),
#     'artist_pop': np.asarray([51.0]),
#     'artist_followers':np.asarray([205331.0]),
#     'artist_genres': np.asarray([b"'neon pop punk', 'pop punk'"]),
#     # 'test': np.asarray([b'test'])
# }
# pprint(can_test_instance)

# Two-Tower Model

## Playlist (query) Tower

In [18]:
EMBEDDING_DIM = 32
PROJECTION_DIM = 5
SEED = 1234
USE_CROSS_LAYER=True
DROPOUT='False'
DROPOUT_RATE='0.33'

In [19]:
# vocab_dict_load["name"]
len(vocab_dict_load["name"])

74028

In [20]:
vocab_dict_load

{'name': array([b'! 2017 Songs', b'! <3', b'! DJ', ...,
        b'\xf0\x9f\xa6\x84\xf0\x9f\xa6\x84',
        b'\xf0\x9f\xa6\x84\xf0\x9f\xa6\x84\xf0\x9f\xa6\x84',
        b'\xf0\x9f\xa6\x8b\xf0\x9f\xa6\x8b\xf0\x9f\xa6\x8b'], dtype=object),
 'artist_name_can': array([b'!!!', b'!Dela Dap', b'!Distain', ..., b'\xed\x9b\x88 Hun',
        b'\xef\xbc\x92\xef\xbc\x98\xef\xbc\x91\xef\xbc\x94',
        b'\xef\xbc\xad\xef\xbc\xa9\xef\xbc\xb9\xef\xbc\xa1\xef\xbc\xb6\xef\xbc\xa9'],
       dtype=object),
 'track_uri_can': array([b'spotify:track:0000uJA4xCdxThagdLkkLR',
        b'spotify:track:0002yNGLtYSYtc0X6ZnFvp',
        b'spotify:track:00039MgrmLoIzSpuYKurn9', ...,
        b'spotify:track:7zzwsf6tmZETUV26kR7D5z',
        b'spotify:track:7zzxEH0xUl5k3p6IxUfgAO',
        b'spotify:track:7zzyrYnZIfvYAGwl7lRb7X'], dtype=object),
 'artist_uri_can': array([b'spotify:artist:0001ZVMPt41Vwzt1zsmuzp',
        b'spotify:artist:0001cekkfdEBoMlwVQvpLg',
        b'spotify:artist:0001wHqxbF2YYRQxGdbyER', ...,

In [64]:
class Playlist_Model(tf.keras.Model):
    def __init__(self, layer_sizes, vocab_dict):
        super().__init__()

        # ========================================
        # non-sequence playlist features
        # ========================================
        
        # Feature: playlist name
        self.pl_name_text_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.TextVectorization(
                    max_tokens=MAX_TOKENS, # not needed if passing vocab
                    # vocabulary=vocab_dict['name'], 
                    name="pl_name_txt_vectorizer", 
                    ngrams=2
                ),
                tf.keras.layers.Embedding(
                    input_dim=MAX_TOKENS + 1,
                    output_dim=EMBEDDING_DIM,
                    mask_zero=False,
                    name="pl_name_emb_layer",
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="pl_name_pooling"),
            ], name="pl_name_emb_model"
        )
        
        # Feature: collaborative
        collaborative_vocab = np.array([b'false', b'true'])
        
        self.pl_collaborative_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.StringLookup(
                    vocabulary=collaborative_vocab, 
                    mask_token=None, 
                    name="pl_collaborative_lookup", 
                    output_mode='int'
                ),
                tf.keras.layers.Embedding(
                    input_dim=len(collaborative_vocab) + 1,
                    output_dim=EMBEDDING_DIM,
                    mask_zero=False,
                    name="pl_collaborative_emb_layer",
                ),
            ], name="pl_collaborative_emb_model"
        )
        
        # Feature: pid
        self.pl_track_uri_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.StringLookup(
                    vocabulary=tf.constant(vocab_dict['track_uri_can']), 
                    mask_token=None, 
                    name="pl_track_uri_lookup", 
                ),
                tf.keras.layers.Embedding(
                    input_dim=len(vocab_dict['track_uri_can'])+1,
                    output_dim=EMBEDDING_DIM,
                    mask_zero=False,
                    name="pl_track_uri_layer",
                ),
            ], name="pl_track_uri_emb_model"
        )
        
        # Feature: n_songs_pl
        # TODO: Noramlize or Descritize?
        n_songs_pl_buckets = np.linspace(
            vocab_dict['min_n_songs_pl'], 
            vocab_dict['max_n_songs_pl'], 
            num=100
        )
        self.n_songs_pl_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.Discretization(n_songs_pl_buckets.tolist()),
                tf.keras.layers.Embedding(
                    input_dim=len(n_songs_pl_buckets) + 1, 
                    output_dim=EMBEDDING_DIM, 
                    name="n_songs_pl_emb_layer",
                )
            ], name="n_songs_pl_emb_model"
        )
        
        # Feature: num_artists_pl
        # TODO: Noramlize or Descritize?
        n_artists_pl_buckets = np.linspace(
            vocab_dict['min_n_artists_pl'], 
            vocab_dict['max_n_artists_pl'], 
            num=100
        )
        self.n_artists_pl_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.Discretization(n_artists_pl_buckets.tolist()),
                tf.keras.layers.Embedding(
                    input_dim=len(n_artists_pl_buckets) + 1, 
                    output_dim=EMBEDDING_DIM, 
                    name="n_artists_pl_emb_layer",
                    mask_zero=False
                )
            ], name="n_artists_pl_emb_model"
        )

        # Feature: num_albums_pl
        n_albums_pl_buckets = np.linspace(
            vocab_dict['min_n_albums_pl'], 
            vocab_dict['max_n_albums_pl'],
            num=100
        )
        self.n_albums_pl_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.Discretization(n_albums_pl_buckets.tolist()),
                tf.keras.layers.Embedding(
                    input_dim=len(n_albums_pl_buckets) + 1, 
                    output_dim=EMBEDDING_DIM, 
                    name="n_albums_pl_emb_layer",
                )
            ], name="n_albums_pl_emb_model"
        )
        
        # ========================================
        # sequence playlist features
        # ========================================
        
        # Feature: artist_name_pl
        self.artist_name_pl_embedding = tf.keras.Sequential(
            [
                # tf.keras.layers.Flatten(),
                tf.keras.layers.StringLookup(
                    vocabulary=vocab_dict['artist_name_pl'], mask_token=''),
                tf.keras.layers.Embedding(
                    input_dim=len(vocab_dict['artist_name_pl']) + 1, 
                    output_dim=EMBEDDING_DIM,
                    name="artist_name_pl_emb_layer",
                    mask_zero=False
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="artist_name_pl_1d"),
            ], name="artist_name_pl_emb_model"
        )
        
        # Feature: track_uri_pl
        # 2.2M unique
        self.track_uri_pl_embedding = tf.keras.Sequential(
            [
                # tf.keras.layers.Flatten(),
                tf.keras.layers.StringLookup(
                    vocabulary=vocab_dict['track_uri_can'], mask_token=''),
                tf.keras.layers.Embedding(
                    input_dim=len(vocab_dict['track_uri_can']) + 1, 
                    output_dim=EMBEDDING_DIM,
                    name="track_uri_pl_emb_layer",
                    mask_zero=False
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="track_uri_1d"),
            ], name="track_uri_pl_emb_model"
        )
        
        # Feature: track_name_pl
        self.track_name_pl_embedding = tf.keras.Sequential(
            [
                # tf.keras.layers.Flatten(),
                tf.keras.layers.StringLookup(
                    vocabulary=vocab_dict['track_name_pl'], 
                    name="track_name_pl_lookup",
                    output_mode='int',
                    mask_token=''
                ),
                tf.keras.layers.Embedding(
                    input_dim=len(vocab_dict['track_name_pl']) + 2, 
                    output_dim=EMBEDDING_DIM,
                    name="track_name_pl_emb_layer",
                    mask_zero=False
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="track_name_pl_1d"),
            ], name="track_name_pl_emb_model"
        )
        
        Feature: duration_ms_songs_pl
        duration_ms_songs_pl_buckets = np.linspace(
            vocab_dict['min_duration_ms_songs_pl'], 
            vocab_dict['max_duration_ms_songs_pl'], 
            num=100
        )
        self.duration_ms_songs_pl_embedding = tf.keras.Sequential(
            [
                # tf.keras.layers.Flatten(),
                tf.keras.layers.Discretization(duration_ms_songs_pl_buckets.tolist()),
                tf.keras.layers.Embedding(
                    input_dim=len(duration_ms_songs_pl_buckets) + 1, 
                    output_dim=EMBEDDING_DIM,
                    name="duration_ms_songs_pl_emb_layer",
                    mask_zero=False
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="duration_ms_songs_pl_emb_layer_pl_1d"),
            ], name="duration_ms_songs_pl_emb_model"
        )
        
        # Feature: album_name_pl
        self.album_name_pl_embedding = tf.keras.Sequential(
            [
                # tf.keras.layers.Flatten(),
                tf.keras.layers.StringLookup(
                    vocabulary=tf.constant(vocab_dict['album_name_pl']), 
                    mask_token=None, 
                    name="album_name_pl_lookup"
                ),
                tf.keras.layers.Embedding(
                    input_dim=len(vocab_dict['album_name_pl']) + 1, 
                    output_dim=EMBEDDING_DIM,
                    name="album_name_pl_emb_layer",
                    mask_zero=False
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="album_name_pl_emb_layer_1d"),
            ], name="album_name_pl_emb_model"
        )
        
        # Feature: artist_pop_pl
        artist_pop_pl_buckets = np.linspace(
            vocab_dict['min_artist_pop'], 
            vocab_dict['max_artist_pop'], 
            num=10
        )
        self.artist_pop_pl_embedding = tf.keras.Sequential(
            [
                # tf.keras.layers.Flatten(),
                tf.keras.layers.Discretization(artist_pop_pl_buckets.tolist()),
                tf.keras.layers.Embedding(
                    input_dim=len(artist_pop_pl_buckets) + 1, 
                    output_dim=EMBEDDING_DIM,
                    name="artist_pop_pl_emb_layer",
                    mask_zero=False
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="artist_pop_1d"),
            ], name="artist_pop_pl_emb_model"
        )
        
        # Feature: artists_followers_pl
        artists_followers_pl_buckets = np.linspace(
            vocab_dict['min_artist_followers'], 
            vocab_dict['max_artist_followers'], 
            num=10
        )
        self.artists_followers_pl_embedding = tf.keras.Sequential(
            [
                # tf.keras.layers.Flatten(),
                tf.keras.layers.Discretization(artists_followers_pl_buckets.tolist()),
                tf.keras.layers.Embedding(
                    input_dim=len(artists_followers_pl_buckets) + 1, 
                    output_dim=EMBEDDING_DIM,
                    name="artists_followers_pl_emb_layer",
                    mask_zero=False
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="artists_followers_pl_1d"),
            ], name="artists_followers_pl_emb_model"
        )
        
        # Feature: track_pop_pl
        track_pop_pl_buckets = np.linspace(
            vocab_dict['min_track_pop'], 
            vocab_dict['max_track_pop'], 
            num=10
        )
        self.track_pop_pl_embedding = tf.keras.Sequential(
            [
                # tf.keras.layers.Flatten(dtype=tf.float32),
                tf.keras.layers.Discretization(track_pop_pl_buckets.tolist()),
                tf.keras.layers.Embedding(
                    input_dim=len(track_pop_pl_buckets) + 1, 
                    output_dim=EMBEDDING_DIM,
                    name="track_pop_pl_emb_layer",
                    mask_zero=False
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="track_pop_pl_1d"),
            ], name="track_pop_pl_emb_model"
        )
        
        # Feature: artist_genres_pl
        self.artist_genres_pl_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.Flatten(),
                tf.keras.layers.StringLookup(
                    vocabulary=vocab_dict['artist_genres_pl'], mask_token=''),
                tf.keras.layers.Embedding(
                    input_dim=len(vocab_dict['artist_genres_pl']) + 2, 
                    output_dim=EMBEDDING_DIM,
                    name="artist_genres_pl_emb_layer",
                    mask_zero=False
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="artist_genres_pl_1d"),
            ], name="artist_genres_pl_emb_model"
        )
        
        # ========================================
        # dense and cross layers
        # ========================================

        # Cross Layers
        if USE_CROSS_LAYER:
            self._cross_layer = tfrs.layers.dcn.Cross(
                projection_dim=PROJECTION_DIM,
                kernel_initializer="glorot_uniform", 
                name="pl_cross_layer"
            )
        else:
            self._cross_layer = None
            
        # Dense Layers
        self.dense_layers = tf.keras.Sequential(name="pl_dense_layers")
        initializer = tf.keras.initializers.GlorotUniform(seed=SEED)
        
        # Use the ReLU activation for all but the last layer.
        for layer_size in layer_sizes[:-1]:
            self.dense_layers.add(
                tf.keras.layers.Dense(
                    layer_size, 
                    activation="relu", 
                    kernel_initializer=initializer,
                )
            )
            if DROPOUT:
                self.dense_layers.add(tf.keras.layers.Dropout(DROPOUT_RATE))
                
        # No activation for the last layer
        for layer_size in layer_sizes[-1:]:
            self.dense_layers.add(
                tf.keras.layers.Dense(
                    layer_size, 
                    kernel_initializer=initializer
                )
            )
        ### ADDING L2 NORM AT THE END
        self.dense_layers.add(
            tf.keras.layers.Lambda(
                lambda x: tf.nn.l2_normalize(
                    x, 1, epsilon=1e-12, name="normalize_dense"
                )
            )
        )
        
    # ========================================
    # call
    # ========================================
    def call(self, data):
        '''
        The call method defines what happens when
        the model is called
        '''
        
        all_embs = tf.concat(
            [
                self.pl_name_text_embedding(data['name']),
                self.pl_collaborative_embedding(data['collaborative']),
                self.pl_track_uri_embedding(data["track_uri_can"]),
                self.n_songs_pl_embedding(data["n_songs_pl"]),
                self.n_artists_pl_embedding(data['num_artists_pl']),
                self.n_albums_pl_embedding(data["num_albums_pl"]),
                
                # sequence features
                self.artist_name_pl_embedding(tf.reshape(data["artist_name_pl"], (-1, MAX_PLAYLIST_LENGTH))), #reshape to get [BATCH, MAX_SEQ_LEN]
                self.track_uri_pl_embedding(tf.reshape(data["track_uri_pl"], (-1, MAX_PLAYLIST_LENGTH))),
                self.track_name_pl_embedding(tf.reshape(data["track_name_pl"], (-1, MAX_PLAYLIST_LENGTH))),
                self.duration_ms_songs_pl_embedding(tf.reshape(data["duration_ms_songs_pl"], (-1, MAX_PLAYLIST_LENGTH))),
                self.album_name_pl_embedding(tf.reshape(data["album_name_pl"], (-1, MAX_PLAYLIST_LENGTH))),
                self.artist_pop_pl_embedding(tf.reshape(data["artist_pop_pl"], (-1, MAX_PLAYLIST_LENGTH))),
                self.artists_followers_pl_embedding(tf.reshape(data["artists_followers_pl"], (-1, MAX_PLAYLIST_LENGTH))),
                self.track_pop_pl_embedding(tf.reshape(data["track_pop_pl"], (-1, MAX_PLAYLIST_LENGTH))),
                self.artist_genres_pl_embedding(tf.reshape(data["artist_genres_pl"], (-1, MAX_PLAYLIST_LENGTH))),
            ], axis=1)
        
        # Build Cross Network
        if self._cross_layer is not None:
            cross_embs = self._cross_layer(all_embs)
            return self.dense_layers(cross_embs)
        else:
            return self.dense_layers(all_embs)

### test playlist tower

In [22]:
layer_sizes=[64,32]

test_playlist_model = Playlist_Model(layer_sizes,vocab_dict_load)

test_playlist_model.pl_name_text_embedding.layers[0].adapt(parsed_dataset_padded.map(lambda x: x['name']).batch(1000))

print("Adapts complete for name")

Adapts complete for name


In [23]:
test_playlist_model.pl_name_text_embedding.layers[0].get_vocabulary()

['',
 '[UNK]',
 'music',
 'country',
 'party',
 'summer',
 'chill',
 'good',
 'jams',
 'songs',
 'rock',
 'playlist',
 'rap',
 'new',
 'the',
 '2017',
 'my',
 'mix',
 'oldies',
 'vibes',
 'workout',
 'old',
 'throwback',
 '2016',
 'work',
 'car',
 'christmas',
 'up',
 'dance',
 'wedding',
 'road',
 '90s',
 '2015',
 'feels',
 'classic',
 'stuff',
 'lit',
 'love',
 'worship',
 '80s',
 'edm',
 'it',
 'school',
 'out',
 'hop',
 'time',
 'trip',
 'throwbacks',
 'pop',
 'hip',
 'best',
 'happy',
 'gym',
 'classics',
 'hip hop',
 'spring',
 'running',
 'road trip',
 'jamz',
 '17',
 'slow',
 'of',
 'all',
 'tunes',
 'fall',
 'run',
 'classic rock',
 'random',
 'disney',
 'feel',
 'alternative',
 'old school',
 'driving',
 '16',
 'me',
 'for',
 'day',
 'feel good',
 'rb',
 'hype',
 'but',
 'i',
 'pregame',
 'indie',
 'goodies',
 'house',
 'spanish',
 'christian',
 'fun',
 '2',
 'beach',
 'hits',
 'back',
 '2014',
 'good vibes',
 'favorites',
 '3',
 'shower',
 'oldies but',
 'mellow',
 'in',
 'g

In [24]:
vocab_dict_load['name_voacb'] = test_playlist_model.pl_name_text_embedding.layers[0].get_vocabulary()

In [30]:
#test with the source batched datset
batched_dataset = parsed_dataset_padded.batch(2)
for x in batched_dataset.take(1):
    print(test_playlist_model(x))

tf.Tensor(
[[-0.12191597 -0.02955313  0.26741976 -0.19543947 -0.08209568  0.38031173
   0.22954062  0.11182079  0.14448637 -0.01198089 -0.03505188  0.2502452
   0.10176286 -0.00109097  0.05177378 -0.1142177   0.01734834 -0.26544943
   0.2931487   0.09069694 -0.15994911  0.44579393  0.04805622 -0.1678858
  -0.0100677  -0.07310402 -0.07062502  0.11664435  0.0698021  -0.02859749
   0.27798328 -0.14243548]
 [-0.41296196 -0.02142718  0.17761382  0.15274735 -0.04896091  0.5102782
   0.08266084 -0.03581357  0.20971023 -0.09563252  0.03559438  0.1491059
   0.04533963 -0.11260843 -0.09356883 -0.099608   -0.16720319 -0.3448041
   0.14887866  0.04505316 -0.0779938   0.19867907 -0.25701636 -0.1361999
   0.04184773 -0.1288359   0.04295938  0.0744244   0.1968722   0.03109774
   0.02927961 -0.16021407]], shape=(2, 32), dtype=float32)


In [27]:
test_playlist_model.summary(expand_nested=True)

Model: "playlist__model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 pl_name_emb_model (Sequenti  (None, 32)               2368928   
 al)                                                             
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| pl_name_txt_vectorizer (Tex  (None, None)           0         |
| tVectorization)                                               |
|                                                               |
| pl_name_emb_layer (Embeddin  (None, None, 32)       2368928   |
| g)                                                            |
|                                                               |
| pl_name_pooling (GlobalAver  (None, 32)             0         |
| agePooling1D)                                                 |
¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯
 pl_collaborative_emb_model   (10, 32)             

## Track (candidate) Tower 

In [65]:
EMBEDDING_DIM = 32
PROJECTION_DIM = 5
SEED = 1234
USE_CROSS_LAYER=True
DROPOUT='False'
DROPOUT_RATE='0.33'

# candidate_test_instance

In [66]:
class Candidate_Track_Model(tf.keras.Model):
    def __init__(self, layer_sizes, vocab_dict):
        super().__init__()
        
        # ========================================
        # Candidate features
        # ========================================
        
        # Feature: artist_name_can
        self.artist_name_can_text_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.TextVectorization(
                    max_tokens=MAX_TOKENS,
                    # vocabulary=vocab_dict["artist_name_can"],
                    name="artist_name_can_txt_vectorizer",
                    ngrams=2,
                ),
                tf.keras.layers.Embedding(
                    input_dim=MAX_TOKENS+1,
                    output_dim=EMBEDDING_DIM,
                    mask_zero=False,
                    name="artist_name_can_emb_layer",
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="artist_name_can_pooling"),
            ], name="artist_name_can_emb_model"
        )
        
        # Feature: track_name_can
        self.track_name_can_text_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.TextVectorization(
                    max_tokens=MAX_TOKENS,
                    # vocabulary=vocab_dict["track_name_can"],
                    name="track_name_can_txt_vectorizer",
                    ngrams=2,
                ),
                tf.keras.layers.Embedding(
                    input_dim=MAX_TOKENS+1,
                    output_dim=EMBEDDING_DIM,
                    mask_zero=False,
                    name="track_name_can_emb_layer",
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="track_name_can_pooling"),
            ], name="track_name_can_emb_model"
        )
        
        # Feature: album_name_can
        self.album_name_can_text_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.TextVectorization(
                    max_tokens=MAX_TOKENS,
                    # vocabulary=vocab_dict["album_name_can"],
                    name="album_name_can_txt_vectorizer",
                    ngrams=2,
                ),
                tf.keras.layers.Embedding(
                    input_dim=MAX_TOKENS+1,
                    output_dim=EMBEDDING_DIM,
                    mask_zero=False,
                    name="album_name_can_emb_layer",
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="album_name_can_pooling"),
            ], name="album_name_can_emb_model"
        )
        
        # Feature: artist_uri_can
        self.artist_uri_can_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.Hashing(num_bins=200_000),
                tf.keras.layers.Embedding(
                    input_dim=200_000+1, 
                    output_dim=EMBEDDING_DIM,
                    name="artist_uri_can_emb_layer",
                ),
            ], name="artist_uri_can_emb_model"
        )
        
        # Feature: track_uri_can
        self.track_uri_can_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.Hashing(num_bins=200_000),
                tf.keras.layers.Embedding(
                    input_dim=200_000+1, 
                    output_dim=EMBEDDING_DIM,
                    name="track_uri_can_emb_layer",
                ),
            ], name="track_uri_can_emb_model"
        )
        
        # Feature: album_uri_can
        self.album_uri_can_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.Hashing(num_bins=200_000),
                tf.keras.layers.Embedding(
                    input_dim=200_000+1, 
                    output_dim=EMBEDDING_DIM,
                    name="album_uri_can_emb_layer",
                ),
            ], name="album_uri_can_emb_model"
        )
        
        # Feature: duration_ms_can
        self.duration_ms_can_normalized = tf.keras.layers.Normalization(
            mean=vocab_dict['avg_duration_ms_songs_pl'],
            variance=vocab_dict['var_duration_ms_songs_pl'],
            axis=None
        )
        
        # Feature: track_pop_can
        self.track_pop_can_normalized = tf.keras.layers.Normalization(
            mean=vocab_dict['avg_track_pop'],
            variance=vocab_dict['var_track_pop'],
            axis=None
        )
        
        # Feature: artist_pop_can
        self.artist_pop_can_normalized = tf.keras.layers.Normalization(
            mean=vocab_dict['avg_artist_pop'],
            variance=vocab_dict['var_artist_pop'],
            axis=None
        )
        
        # Feature: artist_followers_can
        self.artist_followers_can_normalized = tf.keras.layers.Normalization(
            mean=vocab_dict['avg_artist_followers'],
            variance=vocab_dict['var_artist_followers'],
            axis=None
        )
        
        # Feature: artist_genres_can
        self.artist_genres_can_text_embedding = tf.keras.Sequential(
            [
                tf.keras.layers.TextVectorization(
                    max_tokens=MAX_TOKENS,
                    # vocabulary=vocab_dict["artist_genres_can"],
                    name="artist_genres_can_txt_vectorizer",
                    ngrams=2,
                ),
                tf.keras.layers.Embedding(
                    input_dim=MAX_TOKENS+1,
                    output_dim=EMBEDDING_DIM,
                    mask_zero=False,
                    name="artist_genres_can_emb_layer",
                ),
                tf.keras.layers.GlobalAveragePooling1D(name="artist_genres_can_pooling"),
            ], name="artist_genres_can_emb_model"
        )
        
        # ========================================
        # Dense & Cross Layers
        # ========================================
        
        # Cross Layers
        if USE_CROSS_LAYER:
            self._cross_layer = tfrs.layers.dcn.Cross(
                projection_dim=PROJECTION_DIM,
                kernel_initializer="glorot_uniform", 
                name="can_cross_layer"
            )
        else:
            self._cross_layer = None
        
        # Dense Layer
        self.dense_layers = tf.keras.Sequential(name="candidate_dense_layers")
        initializer = tf.keras.initializers.GlorotUniform(seed=SEED)
        
        # Use the ReLU activation for all but the last layer.
        for layer_size in layer_sizes[:-1]:
            self.dense_layers.add(
                tf.keras.layers.Dense(
                    layer_size, 
                    activation="relu", 
                    kernel_initializer=initializer,
                )
            )
            if DROPOUT:
                self.dense_layers.add(tf.keras.layers.Dropout(DROPOUT_RATE))
                
        # No activation for the last layer
        for layer_size in layer_sizes[-1:]:
            self.dense_layers.add(
                tf.keras.layers.Dense(
                    layer_size, 
                    kernel_initializer=initializer
                )
            )
            
    # ========================================
    # Call Function
    # ========================================
            
    def call(self, data):
        
        all_embs = tf.concat(
            [
                self.artist_name_can_text_embedding(data['artist_name_can']),  
                self.track_name_can_text_embedding(data['track_name_can']),  
                self.album_name_can_text_embedding(data['album_name_can']),  
                self.artist_uri_can_embedding(data['artist_uri_can']),  
                self.track_uri_can_embedding(data['track_uri_can']),  
                self.album_uri_can_embedding(data['album_uri_can']),  
                tf.reshape(self.duration_ms_can_normalized(data["duration_ms_can"]), (-1, 1)), 
                tf.reshape(self.track_pop_can_normalized(data["track_pop_can"]), (-1, 1)),  
                tf.reshape(self.artist_pop_can_normalized(data["artist_pop_can"]), (-1, 1)),  
                tf.reshape(self.artist_followers_can_normalized(data["artist_followers_can"]), (-1, 1)),  
                self.artist_genres_can_text_embedding(data['album_uri_can']),  
            ], axis=1
        )
        
        
        
        # return self.dense_layers(all_embs)
                # Build Cross Network
        if self._cross_layer is not None:
            cross_embs = self._cross_layer(all_embs)
            return self.dense_layers(cross_embs)
        else:
            return self.dense_layers(all_embs)

### Candidate model
Note adapts are done on the unique candidate dataset to save time

In [35]:
layer_sizes=[64,32]

test_can_track_model = Candidate_Track_Model(layer_sizes, vocab_dict_load)

#adapts
test_can_track_model.artist_name_can_text_embedding.layers[0].adapt(parsed_candidate_dataset.map(lambda x: x['artist_name_can']).batch(1000))
test_can_track_model.track_name_can_text_embedding.layers[0].adapt(parsed_candidate_dataset.map(lambda x: x['track_name_can']).batch(1000))
test_can_track_model.album_name_can_text_embedding.layers[0].adapt(parsed_candidate_dataset.map(lambda x: x['album_name_can']).batch(1000))
test_can_track_model.artist_genres_can_text_embedding.layers[0].adapt(parsed_candidate_dataset.map(lambda x: x['artist_genres_can']).batch(1000))

# can_result = test_can_track_model([candidate_test_instance])

# print(f"Shape of can_result: {can_result.shape}")
# can_result

## Validate the output in a small batch 

In [36]:
#test with the source batched datset and candidate dataset
batched_dataset = parsed_candidate_dataset.batch(2)
for x in batched_dataset.take(1):
    print(test_can_track_model(x))

tf.Tensor(
[[-1.16611138e-01 -2.04020873e-01 -1.16053328e-01  5.32517061e-02
   7.53906071e-02 -4.25571978e-01 -1.39279783e-01  1.57434329e-01
  -7.10058585e-02 -2.85662040e-02  5.31206608e-01  2.83476382e-01
  -1.90265477e-01 -2.14888752e-01 -1.26938522e-01 -2.40889877e-01
   5.10127246e-01  4.57332358e-02  1.68353155e-01 -4.51710194e-01
   1.78325117e-01 -4.95814562e-01  2.49489173e-02  4.06476349e-01
  -4.88519043e-01  7.03144372e-02  1.10611647e-01  1.93128213e-02
  -2.63601542e-04  2.42397189e-02  2.28524953e-02 -2.54099429e-01]
 [ 6.88601732e-02 -2.39342123e-01 -1.45322546e-01  3.65444720e-02
   1.55565798e-01  8.12748075e-03  1.65407032e-01  1.02503739e-01
  -3.88229489e-02  2.07624465e-01  3.07005078e-01 -1.08786941e-01
  -6.59863651e-02  1.67001560e-02  8.75131488e-02 -1.87251627e-01
  -5.65811470e-02 -7.63903707e-02  1.17914274e-01 -1.65222943e-01
  -4.16381299e-01 -4.63461429e-01 -2.26131201e-01  1.44475937e-01
  -2.54403561e-01 -5.76799661e-02 -2.67308056e-01  1.24412328e-0

In [36]:
test_can_track_model.summary(expand_nested=True)

Model: "candidate__track__model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 artist_name_can_emb_model (  (None, 32)               9206752   
 Sequential)                                                     
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| artist_name_can_txt_vectori  (None, None)           0         |
| zer (TextVectorization)                                       |
|                                                               |
| artist_name_can_emb_layer (  (None, None, 32)       9206752   |
| Embedding)                                                    |
|                                                               |
| artist_name_can_pooling (Gl  (None, 32)             0         |
| obalAveragePooling1D)                                         |
¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯
 track_name_can_emb_model (S  (None, 32)   

## Combined Model

In [67]:
# %%writefile -a vertex_train/trainer/task.py

class TheTwoTowers(tfrs.models.Model):

    def __init__(self, layer_sizes, vocab_dict_load):
        super().__init__()
        
        self.query_tower = Playlist_Model(layer_sizes, vocab_dict_load)
        
        self.candidate_tower = Candidate_Track_Model(layer_sizes, vocab_dict_load)
        
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=parsed_candidate_dataset.batch(128).cache().map(self.candidate_tower)
            )
        )
        
    def compute_loss(self, data, training=False):
        query_embeddings = self.query_tower(data)
        candidate_embeddings = self.candidate_tower(data)

        return self.task(
            query_embeddings, 
            candidate_embeddings, 
            compute_metrics=not training
        ) # turn off metrics to save time on training

In [85]:
layer_sizes=[64,32]

model = TheTwoTowers(layer_sizes, vocab_dict_load)

model.compile(optimizer=tf.keras.optimizers.Adagrad(0.01))

In [ ]:
#do the adapts
model.query_tower.pl_name_text_embedding.layers[0].adapt(parsed_dataset_padded.map(lambda x: x['name']).batch(10_000))
print("Adapts for name complete")
model.candidate_tower.artist_name_can_text_embedding.layers[0].adapt(parsed_candidate_dataset.map(lambda x: x['artist_name_can']).batch(10_000))
print("Adapts for artist_name_can complete")
model.candidate_tower.track_name_can_text_embedding.layers[0].adapt(parsed_candidate_dataset.map(lambda x: x['track_name_can']).batch(10_000))
print("Adapts for track_name_can complete")
model.candidate_tower.album_name_can_text_embedding.layers[0].adapt(parsed_candidate_dataset.map(lambda x: x['album_name_can']).batch(10_000))
print("Adapts for album_name_can complete")
model.candidate_tower.artist_genres_can_text_embedding.layers[0].adapt(parsed_candidate_dataset.map(lambda x: x['artist_genres_can']).batch(10_000))
print("Adapts for artist_genres_can complete")

In [93]:
pl_name_text_vocab = model.query_tower.pl_name_text_embedding.layers[0].get_vocabulary()
artist_name_can_vocab = model.candidate_tower.artist_name_can_text_embedding.layers[0].get_vocabulary()
album_name_can_vocab = model.candidate_tower.album_name_can_text_embedding.layers[0].get_vocabulary()
artist_genres_can_vocab = model.candidate_tower.artist_genres_can_text_embedding.layers[0].get_vocabulary()
track_name_can_vocab = model.candidate_tower.track_name_can_text_embedding.layers[0].get_vocabulary()

In [94]:
vocab_dict_load[f"{MAX_TOKENS}_tokens"] = {'name':[], 'artist_name_can': [], 'album_name_can': [], 'artist_genres_can': []}
vocab_dict_load[f"{MAX_TOKENS}_tokens"].update({"name": pl_name_text_vocab})
vocab_dict_load[f"{MAX_TOKENS}_tokens"].update({"artist_name_can": artist_name_can_vocab})
vocab_dict_load[f"{MAX_TOKENS}_tokens"].update({"album_name_can": album_name_can_vocab})
vocab_dict_load[f"{MAX_TOKENS}_tokens"].update({"artist_genres_can": artist_genres_can_vocab})
vocab_dict_load[f"{MAX_TOKENS}_tokens"].update({"track_name_can": track_name_can_vocab})

In [95]:
vocab_dict_load.keys()

dict_keys(['name', 'artist_name_can', 'track_uri_can', 'artist_uri_can', 'track_name_can', 'album_uri_can', 'album_name_can', 'artist_genres_can', 'unique_pids', 'artist_name_seed_track', 'artist_uri_seed_track', 'track_name_seed_track', 'track_uri_seed_track', 'album_name_seed_track', 'album_uri_seed_track', 'artist_genres_seed_track', 'description_pl', 'artist_name_pl', 'track_uri_pl', 'track_name_pl', 'album_name_pl', 'artist_genres_pl', 'min_duration_ms_seed_pl', 'max_duration_ms_seed_pl', 'min_n_songs_pl', 'max_n_songs_pl', 'min_n_artists_pl', 'max_n_artists_pl', 'min_n_albums_pl', 'max_n_albums_pl', 'min_artist_pop', 'max_artist_pop', 'min_duration_ms_songs_pl', 'max_duration_ms_songs_pl', 'min_artist_followers', 'max_artist_followers', 'min_track_pop', 'max_track_pop', 'avg_duration_ms_seed_pl', 'var_duration_ms_seed_pl', 'avg_n_songs_pl', 'var_n_songs_pl', 'avg_n_artists_pl', 'var_n_artists_pl', 'avg_n_albums_pl', 'var_n_albums_pl', 'avg_artist_pop', 'var_artist_pop', 'avg_dura

## Upload new adapts for use in training

Notice the new keys `20000_tokens`, `50000_tokens`, and `1000000_tokens`. Each one of these dictionaries inside the original dictionary are available and should be used as vocabularies based on the number of tokens. Note each used `ngrams=2`

In [97]:
vocab_dict_load[f"{MAX_TOKENS}_tokens"]["album_name_can"][:3] #example access and validation

['', '[UNK]', 'the']

In [100]:
### Optional - update vocab files after adapts

BUCKET_NAME = 'spotify-v1'
FILE_PATH = 'vocabs/v2_string_vocabs'
FILE_NAME = 'string_vocabs_v1_20220924-tokens.pkl'

client = storage.Client()

with open(f'{FILE_NAME}', 'wb') as pickle_file:
    pkl.dump(vocab_dict_load, pickle_file, protocol=pkl.HIGHEST_PROTOCOL)

In [ ]:
### Upload the

In [103]:
bucket = client.bucket(BUCKET_NAME)
blob = bucket.blob(FILE_PATH)
blob.upload_from_filename(FILE_NAME)

print(f"{FILE_NAME} uploaded to gs://{BUCKET_NAME}/{FILE_PATH}")

string_vocabs_v1_20220924-tokens.pkl uploaded to gs://spotify-v1/vocabs/v2_string_vocabs


In [104]:
## Quick look at the layers
print("Playlist (query) Tower:")

for i, l in enumerate(model.query_tower.layers):
    print(i, l.name)

Playlist (query) Tower:
0 pl_name_emb_model
1 pl_collaborative_emb_model
2 pl_track_uri_emb_model
3 n_songs_pl_emb_model
4 n_artists_pl_emb_model
5 n_albums_pl_emb_model
6 artist_name_pl_emb_model
7 track_uri_pl_emb_model
8 track_name_pl_emb_model
9 duration_ms_songs_pl_emb_model
10 album_name_pl_emb_model
11 artist_pop_pl_emb_model
12 artists_followers_pl_emb_model
13 track_pop_pl_emb_model
14 artist_genres_pl_emb_model
15 pl_cross_layer
16 pl_dense_layers


In [105]:
print("Track (candidate) Tower:")
for i, l in enumerate(model.candidate_tower.layers):
    print(i, l.name)

Track (candidate) Tower:
0 artist_name_can_emb_model
1 track_name_can_emb_model
2 album_name_can_emb_model
3 artist_uri_can_emb_model
4 track_uri_can_emb_model
5 album_uri_can_emb_model
6 normalization_20
7 normalization_21
8 normalization_22
9 normalization_23
10 artist_genres_can_emb_model
11 can_cross_layer
12 candidate_dense_layers


# Local Training

In [106]:
tf.random.set_seed(42)
shuffle_train = parsed_dataset_padded.shuffle(10_000, seed=42, reshuffle_each_iteration=False)

TRAIN = shuffle_train.batch(128)
VALID = parsed_dataset_padded_valid.batch(128)

In [ ]:
import time    

start_time = time.time()

NUM_EPOCHS = 1
layer_history = model.fit(
    TRAIN,
    validation_data=VALID,
    # validation_freq=5,
    epochs=NUM_EPOCHS,
    # callbacks=tensorboard_cb,
    # verbose=1
)

train_time = time.time() - start_time

print(f"Training for {NUM_EPOCHS} epoch, ran for: {train_time:.0} seconds")
accuracy = layer_history.history["factorized_top_k/top_100_categorical_accuracy"][-1]
print(f"Top 100 categorical accuracy: {accuracy}")

# Local Testing

## Loading SavedModels

In [526]:
candidate_tower_uri = 'gs://spotify-tfrs-dir/v2/run-20220920-210334/candidate_tower'
loaded_candidate_model = tf.saved_model.load(candidate_tower_uri)

### Candidate Model

In [527]:
print(list(loaded_candidate_model.signatures.keys()))

['serving_default']


In [528]:
infer = loaded_candidate_model.signatures["serving_default"]
print(infer.structured_outputs)

{'output_1': TensorSpec(shape=(None, 32), dtype=tf.float32, name='output_1')}


In [530]:
loaded_candidate_model.signatures

_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(*, album_name_can, album_uri_can, artist_followers_can, artist_genres_can, artist_name_can, artist_pop_can, artist_uri_can, duration_ms_can, track_name_can, track_pop_can, track_uri_can) at 0x7F1A85BA2C10>})


In [531]:
predict2 = loaded_candidate_model.signatures['serving_default']
predict2.output_shapes

{'output_1': TensorShape([None, 32])}

In [533]:
parsed_candidate_dataset

<MapDataset element_spec={'album_name_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'album_uri_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'artist_followers_can': TensorSpec(shape=(), dtype=tf.float32, name=None), 'artist_genres_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'artist_name_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'artist_pop_can': TensorSpec(shape=(), dtype=tf.float32, name=None), 'artist_uri_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'duration_ms_can': TensorSpec(shape=(), dtype=tf.float32, name=None), 'track_name_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'track_pop_can': TensorSpec(shape=(), dtype=tf.float32, name=None), 'track_uri_can': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [534]:
embs_iter = parsed_candidate_dataset.batch(1).map(
    lambda data: predict2(
        artist_name_can = data["artist_name_can"],
        track_name_can = data['track_name_can'],
        album_name_can = data['album_name_can'],
        track_uri_can = data['track_uri_can'],
        artist_uri_can = data['artist_uri_can'],
        album_uri_can = data['album_uri_can'],
        duration_ms_can = data['duration_ms_can'],
        track_pop_can = data['track_pop_can'],
        artist_pop_can = data['artist_pop_can'],
        artist_followers_can = data['artist_followers_can'],
        artist_genres_can = data['artist_genres_can']
    )
)

embs = []
for emb in embs_iter:
    embs.append(emb)

In [540]:
print(f"Length of embs: {len(embs)}")
embs[0]

Length of embs: 166827


{'output_1': <tf.Tensor: shape=(1, 32), dtype=float32, numpy=
 array([[ 0.32745814,  0.15835902, -0.62232316, -0.18282643,  0.05425396,
          0.08093273, -0.36878368, -0.6777717 , -0.28926852, -0.44578883,
          0.03769037,  0.2013096 ,  0.01650199, -0.38547963, -0.2734376 ,
          0.16123655,  0.12652676, -0.09455037,  0.3709236 , -0.04336764,
         -0.20310198,  0.21418957, -0.44912496, -0.16798183, -0.21492694,
          0.04033846,  0.14083184,  0.25597924,  0.20490994,  0.18837534,
         -0.15261272,  0.06471566]], dtype=float32)>}

In [541]:
cleaned_embs = [x['output_1'].numpy()[0] for x in embs] #clean up the output

print(f"Length of cleaned_embs: {len(cleaned_embs)}")
cleaned_embs[0]

Length of cleaned_embs: 166827


array([ 0.32745814,  0.15835902, -0.62232316, -0.18282643,  0.05425396,
        0.08093273, -0.36878368, -0.6777717 , -0.28926852, -0.44578883,
        0.03769037,  0.2013096 ,  0.01650199, -0.38547963, -0.2734376 ,
        0.16123655,  0.12652676, -0.09455037,  0.3709236 , -0.04336764,
       -0.20310198,  0.21418957, -0.44912496, -0.16798183, -0.21492694,
        0.04033846,  0.14083184,  0.25597924,  0.20490994,  0.18837534,
       -0.15261272,  0.06471566], dtype=float32)

In [553]:
# clean product IDs
track_uris = [x['track_uri_can'].numpy() for x in parsed_candidate_dataset]
track_uris[0]

b'spotify:track:6KhJeYLg1AimCQjH6ii1Al'

In [554]:
track_uris[0]

b'spotify:track:6KhJeYLg1AimCQjH6ii1Al'

In [557]:
track_uris_cleaned = [str(z).replace("b'","").replace("'","") for z in track_uris]
track_uris_cleaned[0]

'spotify:track:6KhJeYLg1AimCQjH6ii1Al'

In [558]:
print(f"Length of track_uris: {len(track_uris)}")
print(f"Length of track_uris_cleaned: {len(track_uris_cleaned)}")

Length of track_uris: 166827
Length of track_uris_cleaned: 166827


### Write Index Json File

In [559]:
VERSION = 'local-v1'
TIMESTAMP = '092022'

embeddings_index_filename = f'candidate_embeddings_{VERSION}_{TIMESTAMP}.json'

with open(f'{embeddings_index_filename}', 'w') as f:
    for prod, emb in zip(track_uris_cleaned, cleaned_embs):
        f.write('{"id":"' + str(prod) + '",')
        f.write('"embedding":[' + ",".join(str(x) for x in list(emb)) + "]}")
        f.write("\n")

In [564]:
# import json

# with open('candidate_embeddings_local-v1_092022.json', 'r') as f:
#     data = json.load(f)

### Query Model

In [565]:
query_tower_uri = 'gs://spotify-tfrs-dir/v2/run-20220920-210334/query_tower'
loaded_query_model = tf.saved_model.load(query_tower_uri)

In [566]:
loaded_query_model.signatures

_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(*, album_name_can, album_name_pl, album_name_seed_track, album_uri_can, album_uri_seed_track, artist_followers_can, artist_followers_seed_track, artist_genres_can, artist_genres_pl, artist_genres_seed_track, artist_name_can, artist_name_pl, artist_name_seed_track, artist_pop_can, artist_pop_pl, artist_pop_seed_track, artist_uri_can, artist_uri_seed_track, artists_followers_pl, collaborative, description_pl, duration_ms_can, duration_ms_seed_pl, duration_ms_songs_pl, duration_seed_track, n_songs_pl, name, num_albums_pl, num_artists_pl, pid, pos_seed_track, track_name_can, track_name_pl, track_name_seed_track, track_pop_can, track_pop_pl, track_pop_seed_track, track_uri_can, track_uri_pl, track_uri_seed_track) at 0x7F1935582D50>})